In [1]:
# import os
# import sys
# from openai import OpenAI
# from utils import str_to_path

# ROOT_PATH = "C:\\Users\\SHIRAM"
# root_path = str_to_path(path=ROOT_PATH)
# rep_path = str_to_path(path=f"{ROOT_PATH}\\Documents\\markinghelperAI")

import asyncio
import nest_asyncio
from code_reader import respond

nest_asyncio.apply()

async def start_conversation():
    response = respond(
        msg="Give me a nice and simple python code, that prints hello world.",
        history=[],
        file_paths=[],
        model_name='gpt-3.5-turbo',
        max_tokens=100,
        new_system_prompt=None,
        prompt_options=["Code Only"]
    )
    return response

/Users/shriramsunder/Projects/IntotheDeep/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/shriramsunder/Projects/IntotheDeep/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
result = asyncio.run(start_conversation())
result = [ele async for ele in result]

In [3]:
result

['The above code has generated the following output: Hello, World!\n']